## Parte 1

cada pieza se clasifica en una categoría:
- x
- M
- A
- S

cada pieza se manda a un flujo que lo acepta o lo rechaza.

cada regla indica a donde va la pieza si la condición se cumple, las reglas se cumplen en orden, si no se cumple ninguna la última regla se aplica siempre.

ej: ex{x>10:one,m<20:two,a>30:R,A}

In [1]:
import regex as re

In [5]:
def get_input(filename):
    rules = {}
    pieces = []
    separator_seen = False
    
    with open(filename) as file:
        for line in file:
            if line.strip() == '':
                separator_seen = True
                continue
                
            if separator_seen:
                numbers = re.findall('\d+', line)
                pieces.append({x: int(i) for i, x in zip(numbers, ('x', 'm', 'a', 's'))})
            else:
                pos = line.find('{')
                rule_name = line[0: pos]
                rule_body = line[pos + 1: -2]
                rule_parts = rule_body.split(',')
                
                rule = []
                for part in rule_parts:
                    if ':' in part:
                        cond, dest = part.split(':')
                    else:
                        cond = 'True'
                        dest = part
                    rule.append((cond, dest))
                rules[rule_name] = rule
    return rules, pieces

In [9]:
def process_pieces(rules, pieces):
    
    acum = 0
    for piece in pieces:
        cur_rule = 'in'
        x = piece['x']
        m = piece['m']
        a = piece['a']
        s = piece['s']
        
        while cur_rule not in ('A', 'R'):
            for cond, dest in rules[cur_rule]:
                result = eval(cond)
                if result:
                    cur_rule = dest
                    break
                    
            if cur_rule == 'A':
                acum += piece['x'] + piece['m'] + piece['a'] + piece['s']
    return acum        

In [10]:
rules, pieces = get_input('test.txt')
acum = process_pieces(rules, pieces)
acum

19114

In [11]:
rules, pieces = get_input('input.txt')
acum = process_pieces(rules, pieces)
acum

319062

## Parte 2

en lugar de aceptar o rechazar piezas individuales hay que buscar las combinaciones que serán aceptadas, cuantas combinaciones distintas serán aceptadas?

In [12]:
def traverse_rules(rules, pos, cur, ranges):    
    if pos == 'A':
        ranges.append(cur)
        return
    
    if pos == 'R':
        return
    
    extra_conds = []
    for rule in rules[pos]:
        condition = rule[0]
        new_pos = rule[1]
        
        if '<' in condition:
            traverse_rules(rules, new_pos, [c for c in cur] + extra_conds + [condition], ranges)
            parts = condition.split('<')
            new_cond = parts[0] + '>' + str(int(parts[1]) - 1)
            extra_conds.append(new_cond)
        elif '>' in condition:
            traverse_rules(rules, new_pos, [c for c in cur] + extra_conds + [condition], ranges)
            parts = condition.split('>')
            new_cond = parts[0] + '<' + str(int(parts[1]) + 1)
            extra_conds.append(new_cond)
        else:
            traverse_rules(rules, new_pos, [c for c in cur] + extra_conds, ranges)

In [25]:
def count_accepted(ranges):
    cont = 0
    for r in ranges:
        limits = {'x': [1, 4000], 'm': [1, 4000], 'a': [1, 4000], 's': [1, 4000]}
        for cond in r:
            for k in 'xmas':
                if k in cond and '<' in cond:
                    new_val = int(cond.split('<')[1])
                    if limits[k][1] > new_val - 1:
                        limits[k][1] = new_val - 1
                elif k in cond and '>' in cond:
                    new_val = int(cond.split('>')[1])
                    if limits[k][0] < new_val + 1:
                        limits[k][0] = new_val + 1
                        
        acum = 1
        for x in 'xmas':
            acum *= (limits[x][1] - limits[x][0]) + 1
        cont += acum
        
    return cont

In [27]:
rules, _ = get_input('test.txt')
ranges = []
traverse_rules(rules, 'in', [], ranges)
accepted = count_accepted(ranges)
accepted

167409079868000

In [28]:
rules, _ = get_input('input.txt')
ranges = []
traverse_rules(rules, 'in', [], ranges)
accepted = count_accepted(ranges)
accepted

118638369682135